In [1]:
from modules.analitics import HeatGame
from modules.gamesets import Gamesets
from modules.killmethod import Ways_of_killing
from controllers.db_control import CtrlInfo
import controllers.kafka_Infra as kf
import time
import numpy as np
import json

<br>

 - # 1)
<h3> Start Docker Desktop before going on </h3>
<hr>

<br>

 - # 2)
<h3> Run Docker-Compose </h3>

>You should run docker compose to create Apache-Kafka, Zookeeper, and mongoDB

>Make sure that these 3 containers are active before next step. 
<hr>

<br>

 - # 3)
<h3> Prepare Producer </h3>

>This step can be by setting up using terminal, or by coding as a instance from kafka lib.
This example will use terminal to manually run a producer event, and a consumer instance already created will be fed 
<hr>

<br>

#### With every read, run this cell below:

In [2]:
consumer = kf.Consumer_kaf()
consumer.run(broker = 'localhost:19092', topic = 'onlyTopicDone')
time.sleep(2)
mydb = CtrlInfo(r"mongodb://localhost:2717/", 'mycollection')
time.sleep(1)

Assignment: [TopicPartition{topic=onlyTopicDone,partition=%I32d,offset=%s,leader_epoch=%s,error=%s}]


% onlyTopicDone [0] at offset 20 with key None:


Message Log successfully received.
waiting timeout...


In [3]:
dictionary = consumer.get_event()

#offset should be the same in the message above.
offset_kafka = 20
logGames_list = dictionary.get(offset_kafka)

In [4]:
response_gameset = []
response_gameheat = []

for index, content_log in enumerate(logGames_list):
    matchs_commands = {
        'InitGame': 0,
        'ClientConnect': 1,
        'ClientUserinfoChanged': 2,
        'ClientBegin': 3,
        'Kill': 4,
        'ClientDisconnect': 5}
    
    if content_log == []:
        continue
        
    for line in content_log:
        command = 'default'
        line = r"" + line
        for option in matchs_commands.keys():
            if option in line:
                command = matchs_commands.get(option)

        match command:
            case 0: #initGame
                gameset = Gamesets(index, line)
                gameheat = HeatGame(gameset)
                kill_method = Ways_of_killing()

            case 1: # ClientConnect
                try:
                    gameset.clientConnect(line)
                except:
                    pass

            case 2: # ClientUserinfoChanged
                try:
                    gameset.userChangeInfo(line)
                except:
                    pass

            case 3: # ClientBegin
                try:
                    gameset.clientBegin(line)
                except:
                    pass

            case 4: # Kill
                try:
                    gameset.kill(line, kill_method)
                    if not '<world>' in line:
                        gameheat.time_colected(line)
                except:
                    pass

            case 5: # clientdisconnect
                try:
                    gameset.clientdisconnect(line)
                except:
                    pass

            case _:
                continue
    try:
        gamesetInfo = gameset.loadsObj_gameplay(kill_method)
        gameheatInfo = gameheat.loadsObj_analitcs(gameset)
        response_gameset.append(gamesetInfo) 
        response_gameheat.append(gameheatInfo)
    except:
        print(gamesetInfo, '\n', gameheatInfo)
        input()
        pass
    

In [5]:
print(len(response_gameset))
print(len(response_gameheat))

20
20


In [6]:
for slot, _ in enumerate(response_gameset):
    id_inserido1 = mydb.send_to_nosql(response_gameset[slot])
    time.sleep(1)
    id_inserido1_1 = mydb.send_to_nosql(response_gameheat[slot])
    time.sleep(1)

In [7]:
responseDB = mydb.printalldocs()
for doc in responseDB:
    print(doc)

{
    "_id": "653fc06956ee9ae6fe53635b",
    "name": "Lionidas",
    "class": "Spartan",
    "stregth": 80
}
{
    "_id": "653fc08656ee9ae6fe53635c",
    "name": "Lionidas",
    "class": "Spartan",
    "stregth": 80
}
{
    "_id": "65932dd6ad333f32dab0066b",
    "game_1": {
        "Total_kills": 0,
        "Players_online": [
            "Isgalamido"
        ],
        "Players_disconnected: kills_each": {},
        "Kills": {
            "Isgalamido": 0
        },
        "Kill_by_means": {}
    }
}
{
    "_id": "659332c29fc78e292c43f462",
    "game_1": {
        "Total_kills": 0,
        "Players_online": [
            "Isgalamido"
        ],
        "Players_disconnected: kills_each": {},
        "Kills": {
            "Isgalamido": 0
        },
        "Kill_by_means": {}
    }
}
{
    "_id": "659332c39fc78e292c43f463",
    "Out_game_1": {
        "Game map": "q3dm17",
        "Game type": "0",
        "total_kills": 0,
        "Max players logged": 1,
        "ttk (min)": 0,
    

In [9]:
mydb.close_connection()